In [1]:
import pandas as pd
# import datasets
from pyprojroot import here
from pprint import pprint
# from transformers import AutoTokenizer
import yaml
import os
import jsonlines
from datasets import load_dataset
import itertools
from typing import Dict, List

In [27]:
def concat_all_json_files(data_dir:str)->pd.DataFrame:
  df = pd.DataFrame()
  for json_file in os.listdir(data_dir):
     tmp_df = pd.read_json(os.path.join(data_dir, json_file))
     df = pd.concat([df, tmp_df], ignore_index=True)
  return df

# def prepare_finetuning_instruction_output_dataset(df:pd.DataFrame)->List:
#   prompt_template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

#     ### Instruction:
#     {instruction}


#     ### Response:"""

#   # num_examples = len(examples["question"])
#   finetuning_dataset = []
#   for i in range(len(df["instruction"])):
#     question = df["instruction"][i]
#     answer = df["response"][i]
#     text_with_prompt_template = prompt_template.format(question=question)
#     finetuning_dataset.append({"instruction": text_with_prompt_template, "response": answer})

#   return finetuning_dataset

In [60]:
def prepare_finetuning_qa_dataset(df:pd.DataFrame)->List:
  prompt_template = """### Question:
  {question}


  ### Answer:"""

  # num_examples = len(examples["question"])
  finetuning_dataset = []
  for i in range(len(df["question"])):
    question = f"### Question:{df['question'][i]}\n\n\n### Answer:"
    answer = df['answer'][i]
    # question = df["question"][i]
    # answer = df["answer"][i]
    # text_with_prompt_template = prompt_template.format(question=question)
    # finetuning_dataset.append({"question": text_with_prompt_template, "answer": answer})
    finetuning_dataset.append({"question": question, "answer": answer})

  return finetuning_dataset

with open(here("configs/config.yml")) as cfg:
        app_config = yaml.load(cfg, Loader=yaml.FullLoader)
print(os.listdir(here(app_config["json_dir"]["dir"])))
all_q_a_df = concat_all_json_files(here(app_config["json_dir"]["dir"]))
dataset = prepare_finetuning_qa_dataset(all_q_a_df)

print("One datapoint in the finetuning dataset:")
pprint(dataset[0])

['about.json', 'community_engagement.json', 'customer_support.json', 'environmental_sustainability.json', 'marketing_and_communications.json', 'output_qa.json', 'product_specification.json', 'technical_support.json', 'technical_support_qa.json']
One datapoint in the finetuning dataset:
{'answer': 'CubeTriangle aims to revolutionize the way people interact with '
           'technology in their everyday lives by delivering innovative and '
           'user-friendly products for the modern home.',
 'question': '### Question:\n'
             "  What is CubeTriangle's main focus in consumer electronics?\n"
             '\n'
             '\n'
             '  ### Answer:'}


In [45]:
print(all_q_a_df.shape)
pprint(dataset[:4])
all_q_a_df.head(4)

(244, 2)
[{'answer': 'CubeTriangle aims to revolutionize the way people interact with '
            'technology in their everyday lives by delivering innovative and '
            'user-friendly products for the modern home.',
  'question': '### Question:\n'
              "  What is CubeTriangle's main focus in consumer electronics?\n"
              '\n'
              '\n'
              '  ### Answer:'},
 {'answer': 'CubeTriangle is headquartered in Ottawa, Canada, a city known for '
            'its vibrant tech community and innovative spirit.',
  'question': '### Question:\n'
              '  Where is CubeTriangle headquartered?\n'
              '\n'
              '\n'
              '  ### Answer:'},
 {'answer': 'CubeTriangle boasts a diverse team of over 500 employees, '
            'including engineers, designers, marketing experts, and customer '
            'support professionals, all dedicated to pushing the boundaries of '
            'technological innovation.',
  'question': 

,question,answer
0,What is CubeTriangle's main focus in consumer ...,CubeTriangle aims to revolutionize the way peo...
1,Where is CubeTriangle headquartered?,"CubeTriangle is headquartered in Ottawa, Canad..."
2,How many employees does CubeTriangle have?,CubeTriangle boasts a diverse team of over 500...
3,What is CubeTriangle's motto?,CubeTriangle's motto is 'Connecting Innovation...


In [49]:
def prepare_finetuning_qa_dataset(df: pd.DataFrame) -> Dict:
    finetuning_dataset = {"text": {}}
    for i in range(len(df["question"])):
        question = f"### Question: {df['question'][i]}\n\n\n"
        answer = f"### Answer:{df['answer'][i]}"
        
        # Add each pair to the dictionary using the index as the key
        finetuning_dataset["text"][i] = question + answer
    
    return finetuning_dataset

with open(here("configs/config.yml")) as cfg:
        app_config = yaml.load(cfg, Loader=yaml.FullLoader)
print(os.listdir(here(app_config["json_dir"]["dir"])))
all_q_a_df = concat_all_json_files(here(app_config["json_dir"]["dir"]))
dataset = prepare_finetuning_qa_dataset(all_q_a_df)

print("\n\nOne datapoint in the finetuning dataset:\n")
pprint(dataset["text"][0])
print("\n\nNumber of Q&A pairs:", {len(dataset["text"])})

['about.json', 'community_engagement.json', 'customer_support.json', 'environmental_sustainability.json', 'marketing_and_communications.json', 'output_qa.json', 'product_specification.json', 'technical_support.json', 'technical_support_qa.json']


One datapoint in the finetuning dataset:

("### Question: What is CubeTriangle's main focus in consumer electronics?\n"
 '\n'
 '\n'
 '### Answer:CubeTriangle aims to revolutionize the way people interact with '
 'technology in their everyday lives by delivering innovative and '
 'user-friendly products for the modern home.')


Number of Q&A pairs: {244}


In [46]:
from datasets import load_dataset
dataset_name = "timdettmers/openassistant-guanaco"
guanaco_trian = load_dataset(dataset_name, split="train")
guanaco_test = load_dataset(dataset_name, split="test")
print(guanaco_trian)
print(guanaco_test)

c:\Users\farza\anaconda3\envs\AR-RT-LLM\Lib\site-packages\huggingface_hub\repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

c:\Users\farza\anaconda3\envs\AR-RT-LLM\Lib\site-packages\huggingface_hub\repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Dataset({
    features: ['text'],
    num_rows: 9846
})
Dataset({
    features: ['text'],
    num_rows: 518
})


In [50]:
dataset["text"]

{0: "### Question: What is CubeTriangle's main focus in consumer electronics?\n\n\n### Answer:CubeTriangle aims to revolutionize the way people interact with technology in their everyday lives by delivering innovative and user-friendly products for the modern home.",
 1: '### Question: Where is CubeTriangle headquartered?\n\n\n### Answer:CubeTriangle is headquartered in Ottawa, Canada, a city known for its vibrant tech community and innovative spirit.',
 2: '### Question: How many employees does CubeTriangle have?\n\n\n### Answer:CubeTriangle boasts a diverse team of over 500 employees, including engineers, designers, marketing experts, and customer support professionals, all dedicated to pushing the boundaries of technological innovation.',
 3: "### Question: What is CubeTriangle's motto?\n\n\n### Answer:CubeTriangle's motto is 'Connecting Innovation, Transforming Lifestyles.'",
 4: "### Question: What is the philosophy behind CubeTriangle's product manufacturing?\n\n\n### Answer:Cube

In [54]:
converted_guanaco_test_dict = {}
guanaco_test_dict = {}

for i, entry in enumerate(guanaco_test):
    guanaco_test_dict[i] = entry["text"]

converted_guanaco_test_dict["text"] = guanaco_test_dict

In [55]:
converted_guanaco_test_dict

{'text': {0: '### Human: Напиши функцию на языке swift, которая сортирует массив целых чисел, а затем выводит его на экран### Assistant: Вот функция, которая сортирует массив целых чисел и выводит его на экран:\n\n```swift\nfunc sortAndPrintArray(_ array: [Int]) {\n  // Создаем копию массива, чтобы не изменять исходный\n  var sortedArray = array\n  // Сортируем массив по возрастанию\n  sortedArray.sort()\n  // Выводим отсортированный массив на экран\n  print(sortedArray)\n}\n```\n\n\nТы можешь проверить работу функции, вызвав ее с любым массивом целых чисел, например:\n\n```swift\nsortAndPrintArray([5, 2, 7, 9, -3])\n```\n\nНа экране появится:\n\n```sh\n[-3, 2, 5, 7, 9]\n```\n---\nНадеюсь, что я смог тебе помочь. Нужно ли тебе что-нибудь ещё? 😊',
  1: '### Human: Inventa un monstruo altamente complejo, descríbelo y enumera sus ventajas y desventajas ante el ambiente en el que se supone que se debe de encontrar.### Assistant: Aquí te presento mi creación de un monstruo altamente complej

In [ ]:
from transformers import AutoTokenizer
model_name = 'aisquared/dlite-v2-1_5b'
tokenizer = AutoTokenizer.from_pretrained(model_name)
print()
tokenizer.decode(tokenizer.pad_token_id)

(244, 2)


,question,answer
0,What is CubeTriangle's main focus in consumer ...,CubeTriangle aims to revolutionize the way peo...
1,Where is CubeTriangle headquartered?,"CubeTriangle is headquartered in Ottawa, Canad..."
2,How many employees does CubeTriangle have?,CubeTriangle boasts a diverse team of over 500...
3,What is CubeTriangle's motto?,CubeTriangle's motto is 'Connecting Innovation...


[{'question': "### Question:\n  What is CubeTriangle's main focus in consumer electronics?\n\n\n  ### Answer:",
  'answer': 'CubeTriangle aims to revolutionize the way people interact with technology in their everyday lives by delivering innovative and user-friendly products for the modern home.'},
 {'question': '### Question:\n  Where is CubeTriangle headquartered?\n\n\n  ### Answer:',
  'answer': 'CubeTriangle is headquartered in Ottawa, Canada, a city known for its vibrant tech community and innovative spirit.'},
 {'question': '### Question:\n  How many employees does CubeTriangle have?\n\n\n  ### Answer:',
  'answer': 'CubeTriangle boasts a diverse team of over 500 employees, including engineers, designers, marketing experts, and customer support professionals, all dedicated to pushing the boundaries of technological innovation.'},
 {'question': "### Question:\n  What is CubeTriangle's motto?\n\n\n  ### Answer:",
  'answer': "CubeTriangle's motto is 'Connecting Innovation, Transfor

In [16]:
with jsonlines.open(here('data/processed/cube_triangle_q_a_processed.jsonl'), 'w') as writer:
    writer.write_all(dataset)

In [20]:
finetuning_dataset_path = "lamini/lamini_docs"
finetuning_dataset = datasets.load_dataset(finetuning_dataset_path)
print(finetuning_dataset)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1260 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/140 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 140
    })
})


More datasets

In [29]:
datasets = load_dataset("squad")

In [30]:
datasets["train"][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [ ]:
finetuning_dataset_path = "lamini/lamini_docs"
finetuning_dataset = datasets.load_dataset(finetuning_dataset_path)
print(finetuning_dataset)